# Exploring the Topology of Pegasus
D-Wave's newest quantum computer, Advantage, introduces a quantum processing unit (QPU) with a new architecture: the Pegasus family of topologies. This notebook explains the Pegasus topology and how it enables superior performance to previous generations of quantum computers. 
    
1. [The Pegasus Advantage](#The-Pegasus-Advantage) demonstrates and explains the differences in performance between the previous and new architecture for embedding problems on the QPU.
2. [Navigating the Topology](#Navigating-the-Topology) describes the new topology and presents Ocean tools that help you use it.
3. [Example Problem: RAN-K](#Example-Problem:-RAN-K) solves a hard problem on Advantage and DW-2000Q quantum computers.

This notebook should familiarize you with the Pegasus topology and the tools to use it.

<img src="images/anim.gif" width=200x/>

**New to Jupyter Notebooks?** JNs are divided into text or code cells. Pressing the **Run** button in the menu bar moves to the next cell. Code cells are marked by an "In: \[\]" to the left; when run, an asterisk displays until code completion: "In: \[\*\]".

# The Pegasus Advantage

The Advantage system is distinct from all previous generations of quantum computers in being the first to employ the new architecture made possible by recent advances in QPU fabrication technology. 

D-Wave QPUs are lattices of interconnected qubits. While some qubits connect to others via couplers, QPUs are not fully connected. Instead, qubits interconnect in a topology: Chimera for the DW-2000Q and Pegasus for the Advantage.

QPU topology is crucial to *embedding* problems onto quantum computer systems.

## Minor-Embedding: Mapping Problems to Qubits

<div class="alert alert-warning" role="alert" style="margin: 10px">Note: If you already understand how problems are mapped to the D-Wave system, please skip ahead to the next text cell by clicking the **Run** button.</div>
 
D-Wave systems solve binary quadratic models (BQM), the [Ising model](#https://en.wikipedia.org/wiki/Ising_model) traditionally used in statistical mechanics and its computer-science equivalent, the quadratic unconstrained binary optimization ([QUBO](#https://en.wikipedia.org/wiki/Quadratic_unconstrained_binary_optimization)) problem. Given $N$ variables $x_1,...,x_N$, where each variable $x_i$ can have binary values $0$ or $1$, the system finds assignments of values that minimize,

$\sum_i^N q_ix_i + \sum_{i<j}^N q_{i,j}x_i  x_j$,

where $q_i$ and $q_{i,j}$ are configurable (linear and quadratic) coefficients. 

Such objective functions can be represented by graphs. A graph comprises a collection of nodes, representing variables, and edges representing the connections between them. For example, a Boolean AND, $z \Leftrightarrow x_1 \wedge x_2$, expressed in [QUBO formulation](#https://docs.dwavesys.com/docs/latest/c_handbook_3.html) as, 

$x_1 x_2 - 2(x_1+x_2)z +3z$, 

is represented by the graph:

<img src="images/embedding_and.png" width=300x/>

To formulate a problem for the D-Wave system, by programing $q_i$ and $q_{i,j}$ so that assignments of $x_1,...,x_N$ also represent solutions to the problem, requires that the problem graph be mapped to the QPU. [Minor embedding](#https://docs.ocean.dwavesys.com/en/stable/concepts/embedding.html#embedding-sdk) maps problem variables ($x_1, x_2, z$ for the AND gate) to the indexed qubits of the D-Wave QPU, with node values represented by qubit biases and edge values by coupler strengths.

Were the QPU topology fully connected&mdash;a fully-connected graph, or clique, is one in which every node shares an edge with every other node, in this case every qubit would be coupled to every other qubit&mdash;you could simply map each problem variable (graph node) to a qubit and each quadratic interaction (graph edge) to a coupler. But with sparser QPU topologies, you might need to represent some variables as *chains* of two or more qubits in order to couple their representative qubits. 

For example, the Boolean AND is represented by a fully connected graph of three nodes, $K_3$, shown on the left side of the figure below. It cannot be mapped directly to qubits in a Chimera topology. Instead, the graph is mapped to four qubits that are coupled so two qubits (e.g., 0 and 4 in this example) form a chain to represent one variable (here $z$), as shown in the middle and right parts of the figure.

<img src="images/embedding_chimera_and.png" width=500x/>

The strength of couplers between qubits in a chain is set to correlate the qubits strongly, so that in most solutions they have a single value for the variable they represent. 

Ocean software provides tools that handle minor-embedding. One instance of minor-embedding a $K_3$ problem on a D-Wave 2000Q is displayed below using the Ocean software's [dwave-inspector](#https://docs.ocean.dwavesys.com/en/stable/docs_inspector/sdk_index.html) tool. The problem graph, shown on the left, is embedded in four qubits , shown on the right against a background of the Chimera topology. The variable highlighted in dark magenta is represented by two qubits, numbers 251 and 253 in this particular embedding.

<img src="images/embedding_3var4qubits.png" width=500x/>

The size and complexity of problems that can be submitted to a quantum computer depends on the QPU's *working graph*, the set of qubits and couplers that are available for computation. Adding qubits expands the QPU's range, obviously, but  denser connectivity reduces the wasting of qubits in representing single variables with chains of multiple qubits. Long chains can also reduce solution quality. 

The Pegasus topology enables the Advantage QPU to more than double the number of available qubits compared to the DW-2000Q, and the working graph is denser, meaning each qubit is coupled to a greater number of neighboring qubits. 

## More Qubits and Denser Connectivity
When looking at summary differences between generations of QPUs, those among us who do not spend much of their time minor-embedding problems on QPUs tend to notice mostly the increase in number of qubits. A DW-2000Q has over 2000 while the Advantage has over 5000. That increase is important; however, this subsection will show why it's worth paying attention to connectivity.   

The subsection also introduces useful Ocean software tools that let you work with the QPU topology locally on your computer. Unless you are solving a problem, as section [Example Problem: RAN-K](#Example-Problem:-RAN-K) does, you can explore the topology without needing to access the system itself.

[dwave_networkx](#https://docs.ocean.dwavesys.com/en/stable/docs_dnx/sdk_index.html) is an extension of [NetworkX](#http://networkx.github.io/)&mdash;a Python language package for exploration and analysis of networks and network algorithms&mdash;for D-Wave systems. It provides tools for working with Chimera and Pegasus graphs and implementations of graph-theory algorithms on the D-Wave system and other binary quadratic model samplers. 

*dwave_networkx* provides functionality to create Chimera and Pegasus lattices of varying sizes. This first code cell creates Chimera and Pegasus graphs of size CN $=$ PN $= 16$. 

The notation CN refers to a Chimera lattice consisting of an $NxN$ grid of *unit cells*. Unit cells are described in more detail below; for now it's sufficient to think of them as vertically and horizontally tiled, sparsely connected, groups of more densely connected qubits. The DW-2000Q has a C16 lattice, meaning $16x16$ unit cells. The Advantage has a P16 lattice that is explored below. 



In [ ]:
import dwave_networkx as dnx

chimera_16 = dnx.chimera_graph(16)
pegasus_16 = dnx.pegasus_graph(16)

print("Qubits in a full working graph: \n    Chimera: {} \n    Advantage: {}".format(len(chimera_16.nodes), len(pegasus_16.nodes)))

The yield of the working graph of most quantum computers is typically a bit lower than the maximum for the lattice size dues to manufacturing variations. These Ocean tools can also generate working graphs that are not full. You will use that capability in this notebook.

*dwave_networkx* also provides functions to plot the topologies. Notice the difference in density of qubits (green dots) and connectivity (black edges) between the two $16x16$ lattices. 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(1, 2, figsize=(14,6))

dnx.draw_chimera(chimera_16, ax=ax[0], node_size=5, node_color='g')
ax[0].set_title('Chimera C16', fontsize=18)

dnx.draw_pegasus(pegasus_16, ax=ax[1], node_size=5, node_color='g')
ax[1].set_title('Pegasus P16', fontsize=18)

## Performance: Embedding Problems in QPUs
The next three code cells import three functions used throughout this section to demonstrate the embedding performance of the Pegasus topology.

* `random_graph` generates a random graph with a specified number of nodes and density of edges.
* `try_embedding` tries to embed random graphs in both the Chimera and Pegasus graphs. 
* `embedding_loop` runs in a loop over the other two functions. 

First, create a random graph of size 100 nodes with 2 edges per node.

In [ ]:
# To see helper functions, select Jupyter File Explorer View from the Online Learning page
from helpers.embed import random_graph
import networkx as nx
import dimod 

G = random_graph(100, 2)

The `try_embedding` function uses one of Ocean's minor-embedding tools to find a minor-embedding for a given problem. Here you can configure the number or attempts, `tries`, this heuristic algorithm makes and a timeout value, `timeout`, for those attempts. 

<div class="alert alert-warning" role="alert" style="margin: 10px">You can also update the imported code (located in the `helpers` folder for this notebook) to use additional configuration parameters, described here: [minorminer](#https://docs.ocean.dwavesys.com/en/stable/docs_minorminer/source/sdk_index.html).</div>

In [ ]:
# To see helper functions, select Jupyter File Explorer View from the Online Learning page
from helpers.embed import try_embedding
import minorminer

topologies_16 = [('Chimera', chimera_16), ('Pegasus', pegasus_16)]

try_embedding(G, target_graphs=topologies_16, timeout=20, tries=2)

The `embedding_loop` function runs a loop over the two previous functions and saves some of the parameters and results in a [pandas](#https://pandas.pydata.org/pandas-docs/stable/index.html) data structure for convenient analysis.

In [ ]:
# To see helper functions, select Jupyter File Explorer View from the Online Learning page
from helpers.embed import embedding_loop
import pandas as pd

embedding_loop(10, 5, topologies_16)

### Sparse Problems
Intuitively, if a straightforward proportionality exists between the number of problem variables a QPU can embed and its number of qubits, sparse graphs should be the place to see it. Such graphs resemble strings of nodes connected head to tail and one can imagine similarly connected strings of qubits. 

A graph where each node has only two edges is such a sparse problem.

The figure below shows embedding results on C16 and P16 topologies for graphs of sizes that range from 50 to 2000 nodes, creating and running on 10 random graph per size, allowing two embedding attempts per graph.  

<img src="images/16x16_problems10_edges2.png" width=400x/>

You can try for yourself in the following cells. Note that repeated embedding calculations on the full-sized topologies can take minutes to run. By default, the code cells below run on quarter-sized topologies to shorten runtime.  

<div class="alert alert-danger" role="alert" style="margin: 10px"><p><b>Important</b>: Loops that run multiple attempts at embeddings on multiple graphs, some of them big, can take minutes to execute. To reduce wait times, by default the code cells below run on quarter-sized C8 and P8 graphs. You can run on the full-sized C16 and P16 by setting the flag `fast_run=False` in the code cells below.</p></div>

<div class="alert alert-warning" role="alert" style="margin: 10px"><p>Note: In this and similar code cells below, you can shorten the runtime by setting a lower value to the `problems` parameter, representing the number of graphs of each size, and/or the list different sizes, `nodes`, set for the loop.</p><p>You can also modify the `embedding_timeout` and `embedding_tries` parameters used to configure the [minorminer](#https://docs.ocean.dwavesys.com/en/stable/docs_minorminer/source/sdk_index.html) embedding heuristic.</p></div>

Show information for the quarter-sized graph used for fast execution. 

In [ ]:
chimera_8 = dnx.chimera_graph(8)
pegasus_8 = dnx.pegasus_graph(8)

topologies_8 = [('Chimera', chimera_8), ('Pegasus', pegasus_8)]

print("Qubits in a quarter-sized working graph: \n    Chimera: {} \n    Advantage: {}".format(
      len(chimera_8.nodes), len(pegasus_8.nodes)))

Try embedding graphs of increasing size in Chimera and Pegasus topologies. Keep an eye on the lengths of chains found for these embeddings.   

In [ ]:
df_columns = ["Nodes", "Edges", "Problem", "Longest Chain"]

num_nodes = [125, 250, 500]   # For C8 & P8
topologies = topologies_8

fast_run = True      # Set to False for C16 & P16
if not fast_run:
    num_nodes = [500, 1000, 2000]      
    topologies = topologies_16

row = []
for nodes in num_nodes:
    row += embedding_loop(nodes=nodes, 
                          edges=2,
                          target_graphs=topologies,
                          #embedding_timeout=60,
                          #embedding_tries=2,
                          #draw_problem = True,
                          #problems = 2
                          )

results = pd.DataFrame(row, columns=df_columns)                       

Tabulate the results. Notice that even for sparse graphs, connectivity plays a crucial role in enabling you to embed large graphs on a QPU.

In [ ]:
results

Increase the density from 2 edges per node to just 3.

The figure below shows embedding results on C16 and P16 topologies for graphs of sizes that range from 50 to 2000 nodes, creating and running on 10 random graph per size, allowing two embedding attempts per graph.  

<img src="images/16x16_problems10_edges3.png" width=400x/>

By default, the code cells below run on quarter-sized topologies to shorten runtime but you can configure the loop as you wish. Pay attention to the chain lengths for the embeddings.  

In [ ]:
num_nodes = [24, 50, 100, 200]   # For C8 & P8
topologies = topologies_8

fast_run = True      # Set to False for C16 & P16
if not fast_run:
    num_nodes = [100, 200, 400, 800]      
    topologies = topologies_16

row = []
for nodes in num_nodes:   
    row += embedding_loop(nodes=nodes, 
                          edges=3,
                          target_graphs=topologies,
                          #embedding_timeout=60,
                          #embedding_tries=2,
                          #draw_problem = True,
                          #problems = 2
                          )
    
results = pd.DataFrame(row, columns=df_columns)    

Tabulate the results. Note the fast growth of chain lengths even for such sparse problems. 

In [ ]:
results

### Dense Problems
Higher connectivity is where the power of the Pegasus topology lies. Increase the density of connectivity for a relatively small, 100-variables problem.

The figure below shows embedding results for graphs of 100 nodes with a number of edges that ranges from 2 to 14, creating and running on 10 random graph per size, allowing two embedding attempts per graph.  

<img src="images/16x16_problems10_edges_x.png" width=400x/>

By default, the code cell below runs on quarter-sized topologies to shorten runtime but you can configure the loop as you wish. 

In [ ]:
topologies = topologies_8

fast_run = True      # Set to False for C16 & P16
if not fast_run:
    topologies = topologies_16

row = []
for edges in [2, 4, 8, 16]:   
    row += embedding_loop(nodes=100, 
                          edges=edges,
                          target_graphs=topologies,
                          #embedding_timeout=60,
                          #embedding_tries=2,
                          #draw_problem = True,
                          #problems = 2                  
                          )
    
results = pd.DataFrame(row, columns=df_columns)

Tabulate and then plot the results.

In [ ]:
results

In [ ]:
# To see helper functions, select Jupyter File Explorer View from the Online Learning page
from helpers.draw import histogram_chains_edges

histogram_chains_edges(results)

The preceding cells have demonstrated two important points: 

* A topology with denser connectivity enables you to scale up your problems (graphs) in terms of both the number of variables (nodes) and the density of the variables' interactions (edges).
* Real-time application must take the computation time of minor-embedding heuristic algorithms into consideration. 

### Cliques: Fully Connected Graphs
Clique embeddings can be very useful. A minor-embedding for a $K_n$ fully connected graph can be used for all minors of that graph. This means that if your application needs to submit a series of problems of up to size $n$ to the QPU, if you have an embedding for the $K_n$ graph on the QPU, you can simply reuse that embedding for all your problems, and save the computation time during the execution of your application.  

As an intuitive example, the $K_3$ minor-embedding explained above can be reused to embed a two-variable problem, $5x_1x_2$, by not setting the values of variable $z$ and its couplers:

<img src="images/embedding_clique_k3_2vars.png" width=500x/>


<div class="alert alert-warning" role="alert" style="margin: 10px">Note that if your problems are sparse, using a clique embedding can be very wasteful, needlessly restricting the number of variables you can scale up to.</div>

#### Embedding in a Single Unit-Cell ($K_{4,4}$)
Many problems have repetitive structures or can be formulated with repeated, small elements; for example, a problem expressed using Boolean elements such as the AND expression above. In such cases, it's advantageous to minor-embed the problem in a way that exploits the lattice structure of the QPU.  

The Chimera topology's simpler structure is based on a $16x16$ grid of *unit cells*, called *C16*, each with four horizontal qubits connected to four vertical qubits via couplers. 

The figure below shows a $3x3$ lattice (a C3) of Chimera unit cells.
<img src="images/chimera.png" width=300x/>
If repeated structures of a problem can be embedded in a unit cell, you might then be able to duplicate the embedding across the lattice's unit cells. 

Try minor-embedding cliques of varying sizes into a Chimera unit cell and its counterpart structure in the Pegasus topology, a $K_{4,4}$  biclique, which is explained in the next section.

In [ ]:
chimera_1 = dnx.chimera_graph(1)
pegasus_k44 = dnx.pegasus_graph(2, node_list=[4, 5, 6, 7, 40, 41, 42, 43])

fig, ax = plt.subplots(1, 2, figsize=(16,8))

dnx.draw_chimera(chimera_1, ax=ax[0], node_size=1000, with_labels=True, node_color='g')
ax[0].set_title('Chimera C1', fontsize=18)

dnx.draw_pegasus(pegasus_k44, ax=ax[1], node_size=1000, with_labels=True, node_color='g')
ax[1].set_title('Pegasus $K_{4,4}$', fontsize=18)

In [ ]:
from dwave.embedding import *

for variables in range(2, 7):
    
    try:
        embedding = chimera.find_clique_embedding(variables, 1, target_edges=chimera_1.edges)
        print("Chimera: embedded {} variables with longest chain of {}.".format(variables, max([len(chain) for chain in embedding.values()])))
    except ValueError:
        print("Chimera: embedding {} variables failed.".format(variables))

    try:
        #embedding = pegasus.find_clique_embedding(variables, target_graph=pegasus_k44)
        embedding = minorminer.find_embedding(nx.complete_graph(variables), pegasus_k44.edges)
        print("Pegasus: embedded {} variables with longest chain of {}.\n".format(variables, max([len(chain) for chain in embedding.values()])))
    except ValueError:
        print("Pegasus: embedding {} variables failed.\n".format(variables))

Below are [dwave-inspector](#https://docs.ocean.dwavesys.com/en/stable/docs_inspector/sdk_index.html) images of a $K_{5,5}$ clique embedded in the Chimera topology and a $K_{6,6}$ clique embedded in the Pegasus topology:

<img src="images/k_55_chimera.png" width=400x/>

<img src="images/k_66_pegasus.png" width=400x/>

#### Largest Cliques

For a given maximum chain length, you can embed a clique of the following sizes in Pegasus and Chimera graphs (for working graphs with 100% yield; typical QPUs have lower yield):

| Chain Length | 1 | 2  | 3  | 4  | 5  | 6  | 7  | 8  | 9  | 10  | 11  | 12  |
|--------------|---|----|----|----|----|----|----|----|----|-----|-----|-----|
| Chimera      | 2 | 4  | 8  | 12 | 16 | 20 | 24 | 28 | 32 | 36  | 40  | 44  | 
| Pegasus      | 4 | 10 | 20 | 30 | 42 | 54 | 66 | 78 | 90 | 102 | 114 | 126 |    



You can mostly leave minor-embedding to Ocean, but sometimes some manual adjustments can enhance the performance for your problem. The next section describes the topology in more detail.   

### Embedding on QPU Working Graphs
In the preceding subsections you embedded onto working graphs representing QPUs with 100% yields. This subsection looks at embedding for real-world QPUs with realistic yields. 

In [ ]:
# TODO

# Navigating the Topology
You've seen the importance of connectivity. The connectivity of a QPU comes from its couplers. A 2000Q QPU has X couplers; an Advantage QPU has Y couplers. 

You have also seen the lattice structure of the Chimera QPU and an example of its relevance for embedding some classes of problems such as those formulated with Boolean gates.

These two are faces of one coin: lattice structure is determined by qubit couplings which are enabled by the structure.  

The Chimera topology is simple to visualize and understanding it goes a long way to grasping the more complex Pegasus topology. 

Qubits in the Chimera topology are “oriented” on the QPU vertically or horizontally, and coupled by one of two categories of couplers:

* *Internal couplers* connect pairs of orthogonal (with opposite orientation) qubits. Each qubit is connected via internal coupling to 4 other qubits.

* *External couplers* connect colinear pairs of qubits&mdash;pairs of parallel qubits in the same row or column.

In the figure below, green circles at the intersections of qubits signify internal couplers; external couplers, shown as connected blue circles, couple vertical qubits to adjacent vertical qubits (not shown in this single-row lattice) and horizontal qubits to adjacent horizontal qubits. The green horizontal qubit in the center couples internally to four vertical qubits, bolded black, in its own unit cell and to the two blue horizontal qubits in adjacent unit cells. 

<img src="images/chimera_couplers.png" width=500x/>

Chimera qubits are characterized as having:

* Nominal length 4&mdash;each qubit is connected to 4 orthogonal qubits through internal couplers.
* Degree 6&mdash;each qubit is coupled to 6 different qubits.

Plot four Chimera unit cells and notice the couplings: internal couplers connecting vertical and horizontal qubits of each cell (short edges) and external couplers connecting similarly oriented qubits of different unit cells.

In [ ]:
chimera_2 = dnx.chimera_graph(2)
    
dnx.draw_chimera(chimera_2, with_labels=True, node_size=500, node_color='g')
plt.show()

When exploiting the underlying structure of the topology it can be helpful to use a coordinate system based on the structure. In the preceding depictions, qubits are shown with an indexical label scheme, but for navigating you might prefer coordinates based on qubit position in a unit cell and the cell's place in the latticed.

For an m-by-n Chimera lattice, connections can be expressed using a node-indexing notation $(i,j,u,k)$ for each node.

* $(i,j)$ indexes the (row, column) of the Chimera tile. $i$ must be between $0$ and $m-1$, inclusive, and $j$ must be between $0$ and $n-1$, inclusive.
* $u=0$ indicates the left-hand nodes in the tile, and $u=1$ indicates the right-hand nodes.
* $k=0,1,…,t-1$ indexes nodes within either the left- or right-hand shores of a tile.

For the C2 Chimera lattice above, show the *Chimera coordinate* of the second vertical and horizontal qubit in each cell:

In [ ]:
for i in range(1, 32, 4):
    print("Qubit {} has Chimera coordinates {}.".format(i, chimera_2.nodes(data=True)[i]['chimera_index']))

Ocean utilities are available to translate between coordinates.

In [ ]:
coords = dnx.chimera_coordinates(2)

i = 13
c = (1, 1, 0, 1)

print("Qubit {} has Chimera coordinates {}.".format(i, coords.linear_to_chimera(i)))
print("Chimera coordinates {} designate qubit {}.".format(c, coords.chimera_to_linear(c)))

Pegasus qubits are also oriented on the QPU vertically or horizontally but they are coupled by one of three categories of couplers:

* Internal couplers connect pairs of orthogonal (with opposite orientation) qubits. Each qubit is connected via internal coupling to 12 other qubits.

  In comparison with Chimera, internal coupling in Pegasus connects each qubit
  to qubits of opposite orientation in repeated substructures that include more
  than a the internal couplings of a single Chimera unit cell.

* External couplers connect vertical qubits to adjacent vertical qubits and horizontal qubits to adjacent horizontal qubits.

* *Odd couplers* connect similarly aligned pairs of qubits in the same Chimera unit cell, a vertical qubit to another vertical qubit and a horizontal qubit to another horizontal qubit.

The figure below provides a helpful way to envision a recurring structure of the Pegasus topology, similar to the unit cells of Chimera: the division of
internal couplings into $K_{4,4}$ bipartite graphs abstracted as three layers of
Chimera lattices. In this abstraction, each qubit forms part, through its
internal couplers, of a Chimera unit cell in one layer (translucent green square) while
additionally coupling to four qubits of a unit cell in a second layer (translucent blue square)
and two qubits each of two units cells in a third layer (translucent pink squares).

<img src="images/pegasus_zlayered_unitcells.png" width=400x/>

Pegasus qubits are characterized as having:

* Nominal length 12.
* Degree 15.

Plot a P2 Pegasus graph.

In [ ]:
pegasus_2 = dnx.pegasus_graph(2)
    
dnx.draw_pegasus(pegasus_2, with_labels=True, node_size=500, node_color='g')
plt.show()

For one random qubit, horizontal qubit 36, plot the internally coupled qubits of each of the three layers of unit cells, plus its oddly coupled qubit. First identify the adjacent (coupled) qubits. 

In [ ]:
print("Qubit 36 is coupled to qubits {}.".format(pegasus_2.adj[36]))

Plot qubit 36 in bold green, the internally coupled qubits in three colors, and the oddly coupled qubit in red.

In [ ]:
horizontal = dnx.pegasus_graph(2, node_list=[36])
odd = dnx.pegasus_graph(2, node_list=[37])
green = dnx.pegasus_graph(2, node_list=[node for node in range(8, 12)])
blue = dnx.pegasus_graph(2, node_list=[node for node in range(12, 16)])
red = dnx.pegasus_graph(2, node_list=[6, 7, 16, 17])

fig, ax = plt.subplots(1, 1, figsize=(10,10))

dnx.draw_pegasus(pegasus_2, ax=ax, with_labels=True, node_size=500, node_color='y')
dnx.draw_pegasus(horizontal, ax=ax, node_size=800, node_color='g')
dnx.draw_pegasus(odd, ax=ax, node_color='r')
dnx.draw_pegasus(green, ax=ax, node_color='lightgreen')
dnx.draw_pegasus(blue, ax=ax, node_color='lightblue')
dnx.draw_pegasus(red, ax=ax, node_color='lightcoral')

The Pegasus coordinates of a node, $(u,w,k,z)$, can be interpreted as:

* $u$: qubit orientation ($0$ is vertical, $1$ is horizontal).
* $w$: orthogonal major offset.
* $k$: orthogonal minor offset.
* $z$: parallel offset.

For this example of qubit 36: 

* $u=1$ for 36 and oddly coupled 37 because they are horizontal and $u=0$ for qubits internally coupled to qubit 36, which are vertical.
* $w$ increases with rightward movement (the horizontal axis is orthogonal to the internally coupled qubits' vertical orientation). A shift in major offset happens between qubit 11 and 12. 
* $k$ increases with rightward movement. The minor increases with each subsequent internally coupled qubit. 
* $z$ increases with downward movement (the zero is at the top lefthand corder).

In [ ]:
for i in list(blue.nodes) + list(green.nodes) + list(red.nodes) + list(odd):
    print("Qubit {} has Pegasus coordinates {}.".format(i, pegasus_2.nodes(data=True)[i]['pegasus_index']))

You can use the conversion utility to become comfortable with these coordinates.

In [ ]:
coords = dnx.pegasus_coordinates(2)

i = 36
c = (1, 1, 1, 0)

print("Qubit {} has Pegsus coordinates {}.".format(i, coords.linear_to_pegasus(i)))
print("Pegasus coordinates {} designate qubit {}.".format(c, coords.pegasus_to_linear(c)))

Pegasus has an additional set of "nice" coordinates, more compatible with Chimera addressing: $(t,y,x,u,k)$ 

* $0<=x<M−1$ 
* $0<=y<M−1$ 
* $0<=u<2$ 
* $0<=k<4$ 
* $0<=t<3$ 

For any given $0<=t0<3$, the subgraph of nodes with $t=t0$ has the structure of chimera $(M-1, M-1, 4)$ with the addition of odd couplers.

Convert qubit 36 to Pegasus nice coordinates:

In [ ]:
i = 36

print("Qubit {} has Pegsus nice coordinates {}.".format(i, coords.linear_to_nice(i)))

In these coordinates, it's easy to see which qubits are part of the qubit 36's (green) unit cell: 

* Set $t=1$ because it is $1$ for qubit 36.
* Set $x,y=0,0$ for the same row and column.
* Set $u=1$ for horizontal and $u=0$ for vertical qubits.
* Iterate on $k$ for $4$ horizontal and $4$ vertical qubits. 

In [ ]:
green_horizontal = [(1, 0, 0, 1, k) for k in range(4)]
green_vertical = [(1, 0, 0, 0, k) for k in range(4)]

for i in green_horizontal + green_vertical:
    print("Qubit {} has Pegasus nice coordinates {}.".format(i, coords.nice_to_linear(i)))

## Example: Manual Embedding of a 16-Qubit Problem
Typically embedding is handled automatically by Ocean software. The purpose of this example is to gain familiarity with the Pegasus topology.

The example problem is the 16-qubit system shown below, which was studied in https://www.nature.com/articles/ncomms2920, where dots are qubits with colors representing bias values and lines representing couplings (couplings weights are $-1$).
 
<img src='images/16q_system.png'>

To study the behavior od single qubits, the requirement here is to embedd the problem in a QPU with each problem qubit represented by a single physical qubit; i.e., without using chains.

### Chimera
This problem is a perfect fit to the Chimera topology. This code cell displays one possible embedding, using qubits 0 to 15. QPUs typically do not have fabrication yields of 100% so it is not guarenteed that a particular system's working graph has all the required qubits and couplers in the first two Chimera unit cells. If not, this same embedding can be shifted to any other adjacent unit cells.    

Start with two adjacent Chimera unit cells.

In [ ]:
chimera_1_2 = dnx.chimera_graph(2, node_list=[node for node in range(16)])    
fig, ax = plt.subplots(1, 1, figsize=(10,5))

dnx.draw_chimera(chimera_1_2, ax=ax, with_labels=True, node_size=500, node_color='g')

As an exercise, the next two code cells select qubits to represent the problem's qubits. The next step will be to couple these qubits by selecting edges. 

<div class="alert alert-success" role="alert" style="margin: 10px"> 
    <p><b>Exercise:</b> In the first exercise code cell below, try filling in the blue, red, and white nodes. Run the second exercise code cell to check your work. You can iterate on these two cells until you get it right.</p><p><b>Open the hidden solution code cell and run it and the second exercise code cell before continuing to subsequent cells.</b></p></div>

In [ ]:
# Exercise code cell #1

nodes_blue = [1, ]
nodes_red = [0, ]
nodes_white = [6, ]

In [ ]:
nodes_blue = [1, 2, 5, 9, 10, 14]
nodes_red = [0, 3, 4, 7, 8, 11, 12, 15]
nodes_white = [6, 13]

In [ ]:
# Exercise code cell #2

red = dnx.chimera_graph(2, node_list=nodes_red, edge_list=[])
blue = dnx.chimera_graph(2, node_list=nodes_blue, edge_list=[])
white = dnx.chimera_graph(2, node_list=nodes_white, edge_list=[])

fig, ax = plt.subplots(1, 1, figsize=(10,5))

dnx.draw_chimera(chimera_1_2, ax=ax, with_labels=True, node_size=500, 
                 node_color='g', style='dotted')

dnx.draw_chimera(red, ax=ax, node_size=500, node_color='r')
dnx.draw_chimera(blue, ax=ax, node_size=500, node_color='b')
dnx.draw_chimera(white, ax=ax, node_size=500, node_color='w')

<div class="alert alert-success" role="alert" style="margin: 10px"> <p><b>Exercise:</b> In the exercise code cell below, try filling in the edges (representing couplers) that connect the qubits. You can either directly set just the `edges` variable to a list of all needed edges or use the proposed lists that divide the nodes by color. You can iterate on these two cells until you get it right.</p><p><b>Open the hidden solution code cell and run it and the second exercise code cell before continuing to subsequent cells.</b></p></div>

In [ ]:
blue_blue = [(1, 5)]
red_blue = [(0, 5)]
red_white= [(3, 6)]
blue_white = [(1, 6)]

edges = blue_blue + red_blue + red_white + blue_white

In [ ]:
blue_blue = [(1, 5), (2, 5), (9, 14), (10, 14)]
red_blue = [(0, 5), (4, 2), (7, 1), (12, 10), (11, 14), (15, 9)]
red_white= [(3, 6), (8, 13)]
blue_white = [(1, 6), (2, 6), (9, 13), (10, 13), (5, 13), (6, 14)]

edges = blue_blue + red_blue + red_white + blue_white

In [ ]:
problem_graph = dnx.chimera_graph(2, node_list=[node for node in range(16)],
                                  edge_list=edges)    

red = dnx.chimera_graph(2, node_list=nodes_red, edge_list=[])
blue = dnx.chimera_graph(2, node_list=nodes_blue, edge_list=[])
white = dnx.chimera_graph(2, node_list=nodes_white, edge_list=[])

fig, ax = plt.subplots(1, 1, figsize=(10,5))

dnx.draw_chimera(problem_graph, ax=ax, with_labels=True, 
                 node_size=500, node_color='g')

dnx.draw_chimera(red, ax=ax, node_size=500, node_color='r')
dnx.draw_chimera(blue, ax=ax, node_size=500, node_color='b')
dnx.draw_chimera(white, ax=ax, node_size=500, node_color='w')

Below is a [dwave-inspector](#https://docs.ocean.dwavesys.com/en/stable/docs_inspector/sdk_index.html) image of this problem embedded in a DW-2000Q QPU:

<img src='images/16qubit_problem_embedding_chimera.png' width=600x>

### Pegasus
One approach to converting the made-for-Chimera problem is to simply map the Chimera embedding to two adjacent Chimera unit cells ($K_{4,4}$ bicliques).

Above for Chimera, you started with two adjacent unit cells; now you want two adjacent $K_{4, 4}$ structures. Start by looking at the Pegasus P2 graph used earlier:

In [ ]:
dnx.draw_pegasus(pegasus_2, with_labels=True, node_size=500, node_color='g')
plt.show()

Notice the three $K_{4,4}$ structures across the diagonal. Use two of those; for example, the middle and right ones. 

As an exercise, the next two code cells select qubits to represent the problem's qubits. The next step will be to couple these qubits by selecting edges. 

<div class="alert alert-success" role="alert" style="margin: 10px"> 
    <p><b>Exercise:</b> In the first exercise code cell below, try filling in the blue, red, and white nodes. Run the second exercise code cell to check your work. You can iterate on these two cells until you get it right.</p><p><b>Open the hidden solution code cell and run it and the second exercise code cell before continuing to subsequent cells.</b></p></div>

In [ ]:
# Exercise code cell #1

nodes_blue = [1, ]
nodes_red = [0, ]
nodes_white = [6, ]

In [ ]:
nodes_blue = [9, 37, 38, 14, 33, 34]
nodes_red = [8, 11, 36, 39, 12, 15, 32, 35]
nodes_white = [10, 13]

In [ ]:
# Exercise code cell #2

problem_nodes = nodes_blue + nodes_red + nodes_white
problem_graph = dnx.pegasus_graph(2, node_list=problem_nodes)

red = dnx.pegasus_graph(2, node_list=nodes_red, edge_list=[])
blue = dnx.pegasus_graph(2, node_list=nodes_blue, edge_list=[])
white = dnx.pegasus_graph(2, node_list=nodes_white, edge_list=[])

fig, ax = plt.subplots(1, 1, figsize=(10,5))

dnx.draw_pegasus(problem_graph, ax=ax, with_labels=True, node_size=500, 
                 node_color='g', style='dotted')

dnx.draw_pegasus(red, ax=ax, node_size=500, node_color='r')
dnx.draw_pegasus(blue, ax=ax, node_size=500, node_color='b')
dnx.draw_pegasus(white, ax=ax, node_size=500, node_color='w')

<div class="alert alert-success" role="alert" style="margin: 10px"> <p><b>Exercise:</b> In the exercise code cell below, try filling in the edges (representing couplers) that connect the qubits. You can either directly set just the `edges` variable to a list of all needed edges or use the proposed lists that divide the nodes by color. You can iterate on these two cells until you get it right.</p><p><b>Open the hidden solution code cell and run it and the second exercise code cell before continuing to subsequent cells.</b></p></div>

In [ ]:
blue_blue = [(9, 37), ]
red_blue = [(9, 36), ]
red_white= [(10, 39), ]
blue_white = [(10, 37), ]

edges = blue_blue + red_blue + red_white + blue_white

In [ ]:
blue_blue = [(9, 37), (9, 38), (14, 33), (14, 34)]
red_blue = [(9, 36), (8, 38), (11, 37), (12, 34), (14, 35), (15, 33)]
red_white= [(10, 39), (13, 32)]
blue_white = [(10, 37), (10, 38), (13, 33), (13, 34), (9, 13), (10, 14)]

edges = blue_blue + red_blue + red_white + blue_white

In [ ]:
problem_graph = dnx.pegasus_graph(2, node_list=problem_nodes,
                                 edge_list=edges)    

red = dnx.pegasus_graph(2, node_list=nodes_red, edge_list=[])
blue = dnx.pegasus_graph(2, node_list=nodes_blue, edge_list=[])
white = dnx.pegasus_graph(2, node_list=nodes_white, edge_list=[])

fig, ax = plt.subplots(1, 1, figsize=(10,5))

dnx.draw_pegasus(problem_graph, ax=ax, with_labels=True, 
                 node_size=500, node_color='g')

dnx.draw_pegasus(red, ax=ax, node_size=500, node_color='r')
dnx.draw_pegasus(blue, ax=ax, node_size=500, node_color='b')
dnx.draw_pegasus(white, ax=ax, node_size=500, node_color='w')

For a full Pegaus P16 QPU, the qubit indices need to be updated.

In [ ]:
nodes_blue = [376, 391, 3196, 556, 571, 3212]
nodes_red = [361, 406, 3181, 3226, 541, 586, 3182, 3227]
nodes_white = [3211, 3197]

problem_nodes = nodes_blue + nodes_red + nodes_white

blue_blue = [(376, 3196), (391, 3196), (556, 3212), (571, 3212)]
red_blue = [(361, 3196), (3181, 391), (3226, 376), (3182, 571), 
            (586, 3212), (3227, 556)]
red_white = [(406, 3211), (541, 3197)]
blue_white = [(376, 3211), (391, 3211), (556, 3197), (571, 3197), 
              (3196, 3197), (3211, 3212)]

edges = blue_blue + red_blue + red_white + blue_white

Now the problem can be embedded on a P16 Pegasus. 

In [ ]:
problem_graph = dnx.pegasus_graph(16, node_list=problem_nodes,
                                  edge_list=edges)    

red = dnx.pegasus_graph(16, node_list=nodes_red, edge_list=[])
blue = dnx.pegasus_graph(16, node_list=nodes_blue, edge_list=[])
white = dnx.pegasus_graph(16, node_list=nodes_white, edge_list=[])

fig, ax = plt.subplots(1, 1, figsize=(10,6))

dnx.draw_pegasus(problem_graph, ax=ax, with_labels=True, 
                 node_size=500, node_color='g')

dnx.draw_pegasus(red, ax=ax, node_size=500, node_color='r')
dnx.draw_pegasus(blue, ax=ax, node_size=500, node_color='b')
dnx.draw_pegasus(white, ax=ax, node_size=500, node_color='w')

Below is a [dwave-inspector](#https://docs.ocean.dwavesys.com/en/stable/docs_inspector/sdk_index.html) image of this problem embedded in an Advantage  QPU:

<img src='images/16qubit_problem_embedding_p16.png' width=800x>

# Example Problem: RAN-K
RANk problems set all linear biases to zero and and quadratic values to $\pm k$ based on a uniform probability choice. The `generate_rank` function generates a RANk problem for a [NetworkX](#http://networkx.github.io/) random graph. 
Note that the size fits the Chimera too

In [ ]:
def generate_rank(k, variables, interactions, draw=True):
    
    G = nx.random_regular_graph(n=variables, d=interactions)
    bqm = dimod.generators.random.ran_r(k, G)
    
    if draw:
        plt.figure(figsize=(7, 7))
        nx.draw_networkx(G, pos=nx.spring_layout(G), with_labels=False, node_size=25) 
        plt.show()
            
    return bqm

bqm = generate_rank(7, 100, 2)

TODO: Plot the problem

## Solver Availability
This subsection checks whether you have access to both generations of solvers

In [ ]:
import os

from dwave.system.samplers import DWaveSampler
from dwave.cloud.exceptions import *

try:
    qpu_advantage = DWaveSampler(solver={'topology__type': 'pegasus', 'qpu': True})
    qpu_2000q = DWaveSampler(solver={'topology__type': 'chimera', 'qpu': True})
    
    qpus = {'Advantage': qpu_advantage, 'DW-2000Q': qpu_2000q}

    print("Connected to Advantage {} and 2000Q {}.".format(qpu_advantage.solver.id, qpu_2000q.solver.id))
except SolverNotFoundError:
    print("Currently a pair of solvers are unavailable for sections comparing QPU technologies. Try those examples later.")


In [ ]:
from dwave.system import EmbeddingComposite

samplers = {qpu: EmbeddingComposite(qpus[qpu]) for qpu in qpus}

## Single Submission
First, simply submit a random problem to both QPUs.

In [ ]:
num_reads = 5000

nodes = 50
m_edges = 40
k = 7

chain_strengths = {"Advantage": 25, "DW-2000Q": 35}   

bqm = generate_rank(k, nodes, m_edges)
    
samplesets = {}
for qpu in qpus:  
    samplesets[qpu] = samplers[qpu].sample(bqm, 
                                 num_reads=num_reads,
                                 auto_scale=True,
                                 return_embedding=True,
                                 chain_strength=chain_strengths[qpu],
                                 answer_mode='raw')       

Compare the solutions:

In [ ]:
def compare_solutions(samplesets):
    "Print lowest and average energies."
    
    best_energies = {qpu: round(samplesets[qpu].first.energy, 3) for qpu in qpus}
    average_energies = {qpu: round(np.average(samplesets[qpu].record.energy), 3) for qpu in qpus}
    
    print("Best energies found: {} (Advantage) and {} (DW-2000Q).".format(best_energies["Advantage"], best_energies["DW-2000Q"]))
    print("Average energies: {} (Advantage) and {} (DW-2000Q).".format(average_energies["Advantage"], average_energies["DW-2000Q"]))
    
compare_solutions(samplesets)

Analyze the embedding:

In [ ]:
def compare_embeddings(samplesets):
    "Print chain statistics."
    
    average_chains = {qpu: round(np.average([len(chain) for chain in samplesets[qpu].info['embedding_context']['embedding'].values()]), 1) for qpu in qpus}
    longest_chains = {qpu: round(max([len(chain) for chain in samplesets[qpu].info['embedding_context']['embedding'].values()]), 1) for qpu in qpus}
    chain_breaks = {qpu: round(100*np.average(samplesets[qpu].record.chain_break_fraction), 1) for qpu in qpus}
    
    print("Average chain lengths: {} (Advantage) and {} (DW-2000Q).".format(average_chains["Advantage"], average_chains["DW-2000Q"]))
    print("Longest chains: {} (Advantage) and {} (DW-2000Q).".format(longest_chains["Advantage"], longest_chains["DW-2000Q"]))
    print("Average chain breaks percentage: {} (Advantage) and {} (DW-2000Q).".format(chain_breaks["Advantage"], chain_breaks["DW-2000Q"]))
    
compare_embeddings(samplesets)

In [ ]:
num_bins = 100

energy_advantage = samplesets['Advantage'].record.energy
energy_2000q = samplesets['DW-2000Q'].record.energy

bins=np.histogram(np.hstack((energy_advantage, energy_2000q)), bins=num_bins)[1]

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.hist(energy_advantage, bins, color='b', alpha=0.4, label='Advantage')
ax.hist(energy_2000q, bins, color='g', alpha=0.4, label='DW-2000Q')


ax.set_xlabel("Energy")
ax.set_ylabel("Samples")
ax.legend()

For a random problem embedded heuristically, the Advantage is likely but not certain to produce better results. 

## Multiple Submissions


The figure below shows a histogram for naively combining results of a large number (fifty) of similar problems: RAN-7 random values assigned to the edges of a randomly generated `nx.random_regular_graph()` graphs of 50 nodes and 40 edges each. 

<img src="images/ran7_50problems.png" width=500x/>

Such a combined histogram is not a statistically meaningful measure. However, because the problems are large and returned solutions similar (about 10% variance in range), it does capture in essence the impression given by looking at plots of all fifty solutions. Below are the first five of the fifty:

<img src="images/ran7_50problems_first5.png" width=600x/>

To get a feel for the differences, try playing with chain strength between the two chips, the values of k, and the problem size 

In [ ]:
from helpers.draw import histogram_energies
import pandas as pd

num_reads = 5000
num_problems = 5

nodes = 50
m_edges = 40
k = 7

chain_strengths = {"Advantage": 25, "DW-2000Q": 35}   

draw_flag = True
for i in range(num_problems):

    print("\nProblem {}".format(i+1))
   
    bqm = generate_rank(k, nodes, m_edges, draw=draw_flag)
    draw_flag = False # Comment this line to plot each problem
       
    samplesets = {}
    for qpu in qpus:  
            samplesets[qpu] = samplers[qpu].sample(bqm, 
                                 num_reads=num_reads,
                                 auto_scale=True,
                                 return_embedding=True,
                                 chain_strength=chain_strengths[qpu],
                                 answer_mode='raw')  
    
    compare_solutions(samplesets)
    compare_embeddings(samplesets)
    histogram_energies(samplesets)

Copyright &copy; D-Wave Systems Inc.